# Conversational Interface - Chatbot with Claude LLM

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

In this notebook, we will build a chatbot using the Foundation Models (FMs) in Amazon Bedrock. For our use-case we use Claude as our FM for building the chatbot.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import boto3
boto3_bedrock = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')

## Chatbot

We use [CoversationChain](https://python.langchain.com/en/latest/modules/models/llms/integrations/bedrock.html?highlight=ConversationChain#using-in-a-conversation-chain) from LangChain to start the conversation. We also use the [ConversationBufferMemory](https://python.langchain.com/en/latest/modules/memory/types/buffer.html) for storing the messages. We can also get the history as a list of messages (this is very useful in a chat model).

Chatbots needs to remember the previous interactions. Conversational memory allows us to do that. There are several ways that we can implement conversational memory. In the context of LangChain, they are all built on top of the ConversationChain.


LangChain provides several classes and functions to make constructing and working with prompts easy. We are going to use the [PromptTemplate](https://python.langchain.com/en/latest/modules/prompts/getting_started.html) class to construct the prompt from a f-string template. 

In [3]:
from langchain import PromptTemplate

claude_prompt = PromptTemplate.from_template("""The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context. If the AI does not know
the answer to a question, it truthfully says it does not know.

Current conversation:
<conversation_history>
{history}
</conversation_history>

Human: {input}

Assistant:
""")

In [4]:
from langchain.chains import ConversationChain
from langchain.llms.bedrock import Bedrock
from langchain.memory import ConversationBufferMemory


cl_llm = Bedrock(
    model_id="anthropic.claude-instant-v1",
    client=boto3_bedrock,
    model_kwargs={"max_tokens_to_sample": 1000},
)
memory = ConversationBufferMemory()
conversation= ConversationChain(
    llm=cl_llm, verbose=False, memory=ConversationBufferMemory(), prompt=claude_prompt
)

#### First Question

Let's ask the model a few questions

In [5]:
print(conversation.predict(input="Give me three tips on how to start a new garden."))

 Here are three tips for starting a new garden:

1. Prepare your soil. Make sure your soil is loose, fertile and weed-free before planting anything. You can have your soil tested to see if any nutrients need to be added. Turning the soil and adding compost or other organic matter is important for healthy plant growth.

2. Choose a mix of sun-loving and shade-tolerant plants. Consider how much sunlight each part of your garden gets throughout the day and select plants that will thrive in those conditions. Some vegetables and flowers need at least 6 hours of direct sun per day. 

3. Start with easy-care plants. If you're just starting your first garden, focus on plants that aren't fragile or highly specific about their growing needs. Good starter plants include carrots, tomatoes, zinnias, marigolds and herbs like parsley, mint and chives. They tend to be relatively pest- and disease-resistant.


#### Build on the questions

Let's ask a question without mentioning the word garden to see if model can understand previous conversation

In [6]:
print(conversation.predict(input="Cool. Will that work with tomatoes?"))

 Yes, those tips should work well for tomatoes. Here are a few additional tips specific to growing tomatoes:

- Tomatoes like full sun, so plant them in the hottest, sunniest spot in your garden where they'll get at least 6 hours of direct sun per day. 

- Enrich the soil with compost or a balanced fertilizer before planting. Tomatoes are heavy feeders and will produce more fruits with rich, nutrient-dense soil. 

- Cage or stake the tomato plants as they grow to keep the heavy fruits off the ground and prevent disease issues. You can use tomato cages, stakes or a vertical cable system.

- Water tomatoes deeply but infrequently, about 1-2 inches per week. Consistent moisture is important but you don't want wet foliage which can lead to diseases. 

- In colder regions, wait to transplant tomatoes outside until nighttime temperatures stay above 50°F to avoid stressing the plants. You can get a head start by starting seeds indoors 6-8 weeks before the last spring frost date.

- Prune suck

#### Finishing this conversation

In [7]:
print(conversation.predict(input="That's all, thank you!"))

 You're very welcome!
